In [1]:
from __future__ import annotations
import uproot
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import plotly
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots
from typing import overload
from abc import ABC, abstractmethod
import numpy as np
import sympy
from sympy.integrals.transforms import laplace_transform
import random
import math
from tqdm.auto import tqdm, trange
from concurrent.futures import ThreadPoolExecutor
import scipy.stats as stats
from scipy.optimize import curve_fit


from Class1 import parametricValue

    
from Class1 import fixedValue


from Class1 import parametricValueNumpy


from Class1 import parametricValueSympy


from Class1 import binning


from Class1 import discretepdf


from Class1 import lumiDist


from Class1 import lumiDistSympyLaplace


from Class1 import lumiDistNumpy


from Class1 import SimulationVSPu



In [2]:
from Class1 import RunInfo


In [3]:
from pathlib import Path

import sys
sys.path.append('../nTuplizer')
from allInfo import GetAllRuns, GetAllEras 
def export_graph(g , n , pname, pdir):
    ghtml = g.to_html()
    Path('{0}/{1}/'.format(pdir , pname) ).mkdir(parents=True, exist_ok=True)
    with open('{0}/{1}/{2}.html'.format(pdir , pname , n) , 'w') as f:
        f.write(ghtml)
def publish_res(rinfo , gchi2 , vname ,  name , publish_dir):
    gg = rinfo.plot_lumi_distribution(subRuns=-1 , colorLumiScale=4 , density=False)
    gg.update_layout(
        title="Distribution of BX luminosity",
        xaxis_title='luminosity mub/25ns',
        yaxis_title="probability",
        legend_title="Runs",
    )
    export_graph(gg , '01-lumi_distribution' , name , publish_dir)
    
    gg = rinfo.plotPUDists(70)
    gg.update_layout(
        title="PU distribution for sigma=70",
        xaxis_title='number of PU',
        yaxis_title="probability",
    )    
    export_graph(gg , '02-pu_distribution' , name , publish_dir)
    
  
    gg = rinfo.plotPredictions([60,65,75,80,90])
    gg.update_layout(
        title="Distribution of BX luminosity",
        xaxis_title='luminosity mub/25ns',
        yaxis_title="probability",
        legend_title="Runs",
    )    
    export_graph(gg , '04-{0}_predictions'.format(vname) , name , publish_dir)
    
    if rinfo.parentRun is None:
        gg = rinfo.plotDataDist(zoom=True)
        export_graph(gg , '03-{0}_distribution'.format(vname) , name , publish_dir)

        for _xsec in [60,65,75,80,90]:
            gg = rinfo.plotRunPredictions(_xsec)
            export_graph(gg , '05-{0}_predictions_xsec{1}'.format(vname , _xsec) , name , publish_dir)
            
        export_graph(gchi2 , '06-bestfit_details'  , name , publish_dir)
    export_graph(rinfo.postFitPlots(), '07-postFit_plots'  , name , publish_dir)
    export_graph(rinfo.pullPlots(50 , 1.3) , '08-pullPlots' , name , publish_dir)
    export_graph(rinfo.NadjiehPullPlots(50 , 1.3) , '08-NadjiehPullPlots' , name , publish_dir)
    
    if rinfo.parentRun is None:
        export_graph(rinfo.aggregateFitRes() , '09-summary1' , name ,publish_dir)
        export_graph(rinfo.aggregateFitRes2() , '09-summary2' , name ,publish_dir)
        export_graph(rinfo.aggregateFitRes3() , '09-summary3' , name ,publish_dir)
        export_graph(rinfo.aggregateFitRes4() , '09-summary4' , name ,publish_dir)
        
def process_and_publish(year , era , vname , vmin , vmax , vnbins , pu_max ,
                        lumiName = 'PHYSICSDel' , lumiQ =  np.array([0,0.004,0.1,.2,0.3,.4,0.5,.6,0.7,.8,0.9,1.0]) , lnbins_per_q = 5 ,
                        xsec = np.arange(40,100,1), nthreads = 30 , publish_dir = '/eos/user/c/cmstandi/www/PU/newres/NimaTest/AllRuns' ):
    
    name = '{0}{1}/{2}'.format(era, year , vname)
    var_bins = np.linspace(vmin,vmax,vnbins+1)
    simDist = SimulationVSPu(vname , year = year , var_bins=var_bins , pu_max=pu_max , nthreads=nthreads)

    g = simDist.plot(param=1)
    simDist.plot(param=10 , g=g)
    simDist.plot(param=20 , g=g)
    simDist.plot(param=30 , g=g)
    simDist.plot(param=40 , g=g)
    simDist.plot(param=50 , g=g)
    simDist.plot(param=60 , g=g)
    simDist.plot(param=70 , g=g)
    simDist.plot(param=80 , g=g)
    simDist.plot(param=90 , g=g)
    g.update_layout(
        title="Simulated distribution for PU",
        xaxis_title=vname,
        yaxis_title="probability",
        legend_title="Pileup values",
    )    
    export_graph(g , '00-Simulation' , name , publish_dir )
    
    allRuns = sorted( list(set( GetAllRuns(year , era) ) ) )
    #allRuns = allRuns[1:2]
    #print(allRuns)
    print('is going to run over {0} runs'.format( len(allRuns)) )
    
    rinfo = RunInfo(0 , vname , var_bins , lumiName ,lumiQ , nbins_perq=lnbins_per_q , sub_runs=allRuns , 
                    nthreads=nthreads , xsecs = xsec )
    
    rinfo.setSimulation(simDist)
    gchi2 = rinfo.fit()

    try:
        publish_res(rinfo , gchi2 , vname , name , publish_dir)
        for sr in rinfo._subRuns : #+ rinfo._subRunsSameLumiBins:
            #ext = '/SRSimilarBinning' if sr._isSecondHand else '/SRSameBinning'
            publish_res(sr , None , vname , vname , '{0}/{1}{2}/Runs/Run{3}'.format(publish_dir , era, year , sr.run) )
    except Exception as e:
        print(e)
        return rinfo
    del rinfo
    del simDist
    return None

In [ ]:
variables = { "nVertices" : ( "nVertices" , 90 , 0 , 90 ) ,
              "nGoodVertices" : ("nGoodVertices", 80, 0 , 80) ,
              "nEles" : ("nEles" , 10 , 0 , 10 ) ,
              "nMus" : ("nMus" , 10 , 0 , 10 ),
              "nChargedHadrons" : ("nChargedHadrons" , 120 , 0 , 1200 ),
              "nLostTracks": ("nLostTracks" , 35 , 0 , 35 ),
              "nPhotons" : ("nPhotons" , 120 , 0 , 600 ),
              "nNeutralHadrons" : ("nNeutralHadrons" , 60 , 0 , 120 ),
              "fixedGridRhoAll" : ("fixedGridRhoAll" , 40 , 0 , 40 ),
              "fixedGridRhoFastjetAll" : ("fixedGridRhoFastjetAll" , 40 , 0 , 40 ),
              "fixedGridRhoFastjetAllCalo" : ("fixedGridRhoFastjetAllCalo" , 25 , 0 , 25 ),
              "fixedGridRhoFastjetCentral" : ("fixedGridRhoFastjetCentral" , 50 , 0 , 50 ),
              "fixedGridRhoFastjetCentralCalo" : ("fixedGridRhoFastjetCentralCalo" , 20 , 0 , 20 ),
              "fixedGridRhoFastjetCentralChargedPileUp" : ("fixedGridRhoFastjetCentralChargedPileUp" , 35 , 0 , 35 ),
              "fixedGridRhoFastjetCentralNeutral" : ("fixedGridRhoFastjetCentralNeutral" , 12 , 0 , 12 ) }

for v,vinfo in variables.items():
    process_and_publish(2018 , 'B' , vinfo[0] , vinfo[2] , vinfo[3] , vinfo[1] , 90  )

  0%|          | 0/92 [00:00<?, ?it/s, SIMULATION]

is going to run over 43 runs


  0%|          | 0/111 [00:00<?, ?it/s, RUN 0]

  0%|          | 0/25 [00:00<?, ?it/s, RUN 317291]

  0%|          | 0/25 [00:00<?, ?it/s, RUN 317391]

...

  0%|          | 0/11 [00:00<?, ?it/s, RUN 317435]

  0%|          | 0/11 [00:00<?, ?it/s, RUN 317182]

  0%|          | 0/11 [00:00<?, ?it/s, RUN 317626]

  0%|          | 0/11 [00:00<?, ?it/s, RUN 317320]

  0%|          | 0/11 [00:00<?, ?it/s, RUN 317392]

In [ ]:
#Rinfo = process_and_publish(2017 , 'D' , 'nGoodVertices' , 0 , 90 , 90 , 90  )

In [ ]:
#Rinfo = process_and_publish(2017 , 'D' , 'fixedGridRhoFastjetAllCalo' , 0 , 30 , 30 , 90  )

In [ ]:
#Rinfo = process_and_publish(2017 , 'D' , 'fixedGridRhoFastjetCentralNeutral' , 0 , 30 , 30 , 90  )

In [ ]:
#Rinfo = process_and_publish(2017 , 'D' , 'nChargedHadrons' , 0 , 1500 , 150 , 90  )

In [ ]:
#Rinfo = process_and_publish(2017 , 'D' , 'nVertices' , 0 , 90 , 90 , 90  )

In [ ]:
simDist = SimulationVSPu('nVertices' , year = 2017 , var_bins= np.linspace(0,100,101) , pu_max=100 , nthreads=30)

In [ ]:
aa = simDist.plotEfficiencies()

In [ ]:
aa.plot()

In [ ]:
import sys
sys.path.append('../nTuplizer')
from allInfo import GetAllRuns, GetAllEras
allRuns = sorted( list(set( GetAllRuns(2017 , 'D') ) ) )[0]
rinfo = RunInfo( , vname , var_bins , lumiName ,lumiQ , nbins_perq=lnbins_per_q , sub_runs=allRuns , 
                    nthreads=nthreads , xsecs = xsec )

In [ ]:
allRuns